# Daily CalSys Checkout

In [ ]:
# User input 
day_obs = 20250908

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime as dt

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time, TimeDelta
from scipy.interpolate import interp1d

# LSST-specific packages
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.resources import ResourcePath
from lsst_efd_client import EfdClient, resample

## Whitelight Checkout Plots

In [ ]:
sequence_names= ["whitelight_u_24_M385L3_source","whitelight_i_39_M730L5_source",
                 "whitelight_i_39_M810L3_source","whitelight_g_6_M455L4_source",
                 "whitelight_g_6_M505L4_source","whitelight_r_57_M565L3_source",
                 "whitelight_r_57_M660L4_source","whitelight_z_20_M850L3_source",
                 "whitelight_z_20_M940L3_source","whitelight_y_10_M940L3_source",
                 "whitelight_y_10_M970L4_source"]

In [ ]:
async def get_electrometer_data(day_obs):
    client = EfdClient('usdf_efd')
    msg_log_topic = "lsst.sal.Electrometer.logevent_largeFileObjectAvailable"
    selections = ["url","id"]
    length_of_time = 60*24*30 #30 days
    date_str = str(day_obs)  
    date_str = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}T23:59:00.00"
    #date_str = "2025-08-20"
    end_time = Time(date_str, format="isot")
    start_time = end_time - TimeDelta(60.* length_of_time, format='sec')
    elec_df = await client.select_time_series(msg_log_topic,selections, start=start_time, end=end_time)
    elec_df['dayobs'] = elec_df["id"].str.split("_").str[2]

    daily_df = elec_df[elec_df["id"].str.contains("BT529", case=False, na=False)].copy()
    daily_df['led_id'] = daily_df["id"].str.extract(r"_(\d+)#")
    day_df = daily_df[daily_df.dayobs == str(day_obs)]
    return elec_df, daily_df, day_df

In [ ]:
elec_df, daily_df, day_df = await get_electrometer_data(day_obs)

In [ ]:
fig, axarr = plt.subplots(3,4, figsize=(20,15))

for (idx, row), ax in zip(day_df.iterrows(), axarr.flatten()):
    filt, led = sequence_names[int(row.led_id)].split('_')[1], sequence_names[int(row.led_id)].split('_')[3]
    try:
        path = ResourcePath('s3://lfa@' + row.url.split('.org/')[1])
        with path.open("rb") as f:
            hdu = fits.open(f)  # use context manager so files close properly
            ax.plot(hdu[1].data['Elapsed Time'], hdu[1].data['Signal'], 'x-', label=f"Mean (std) {np.mean(hdu[1].data['Signal']):.2e} ({np.std(hdu[1].data['Signal']):.2e})")

            ax.set_title(f"Filter: {filt}, LED: {led}")
            ax.set_xlabel("Exposure Time (s)")
            ax.set_ylabel("Photodiode Current")
            ax.legend()
    except:
        print(f"didn't work: {path}")


fig.suptitle(f"DayObs: {day_obs}")
plt.tight_layout()
plt.show()

In [ ]:
sequence_dict = {name: [] for name in sequence_names}
for day in np.unique(daily_df.dayobs):
    df_ = daily_df[daily_df.dayobs == day]

    #get mean time
    seconds = df_.index.hour * 3600 + df_.index.minute * 60 + df_.index.second
    mean_seconds = np.mean(seconds)
    mean_time = (dt.datetime.min + dt.timedelta(seconds=mean_seconds)).time()

    for idx, row in df_.iterrows():
        filt, led = sequence_names[int(row.led_id)].split('_')[1], sequence_names[int(row.led_id)].split('_')[3]
        try:
            path = ResourcePath('s3://lfa@' + row.url.split('.org/')[1])
            with path.open("rb") as f:
                hdu = fits.open(f)
                sequence_dict[sequence_names[int(row.led_id)]].append([day,mean_time,np.mean(hdu[1].data['Signal']),np.std(hdu[1].data['Signal'])])
        except:
            print(f"didn't work: {row.url}")

In [ ]:
def time_to_seconds(dt):
    return dt.hour*3600 + dt.minute*60 + dt.second

fig, axarr = plt.subplots(6,2,sharex=True,figsize=(15,25))
ax = axarr.ravel()
for i, (led_name, values) in enumerate(sequence_dict.items()):
    filt, led = led_name.split('_')[1], led_name.split('_')[3]
    try:
        values = np.vstack(values)
        means = [float(val) for val in values[:,2]]
        stds = [float(val) for val in values[:,3]]
    
        cvals = np.array([time_to_seconds(t) for t in values[:,1]])  # dates = pd.to_datetime(...)
        sc = ax[i].scatter(values[:,0], means, c=cvals, cmap="viridis")
        ax[i].errorbar(values[:,0], means, yerr=stds,fmt='none',ecolor='black',elinewidth=2)
        ax[i].set_title(f"Filter: {filt}, LED: {led}")
        ax[i].set_xticks(values[:,0], values[:,0], rotation=45)
        cbar = plt.colorbar(sc, ax=ax[i])
    
        ticks = cbar.get_ticks()   # grab the auto ticks
        cbar.set_ticks(ticks)      # lock them in
        cbar.set_ticklabels([f"{int(t//3600):02d}:{int((t%3600)//60):02d}" for t in ticks])
        cbar.set_label("Time of day (HH:MM)")
    except:
        print(f'No data for {led_name}')
plt.tight_layout()

## Laser Temperature

In [ ]:
async def get_laser_temp_plots(day_obs):
    client = EfdClient('usdf_efd')
    length_of_time = 60*24 #min * hours
    date_str = str(day_obs)  # '20250908'
    date_str = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"

    end_time = Time(date_str, format="iso")
    start_time = end_time - TimeDelta(60.* length_of_time, format='sec')
    
    
    temp_log_topic = 'lsst.sal.ESS.temperature'
    temp_selections = ['salIndex','location', 'temperatureItem0', 'temperatureItem1', 
                  'temperatureItem2', 'temperatureItem3', 'temperatureItem4',
                  'temperatureItem5', 'temperatureItem6', 'temperatureItem7']
    temp_df = await client.select_time_series(temp_log_topic, temp_selections, start=start_time, end=end_time)
    temp_df = temp_df[temp_df.salIndex == 107]
    if len(temp_df) > 0:
    
        hum_log_topic = 'lsst.sal.ESS.relativeHumidity'
        hum_selections = ['salIndex','location', 'relativeHumidityItem']
        hum_df = await client.select_time_series(hum_log_topic, hum_selections, start=start_time, end=end_time)
        hum_df = hum_df[hum_df.salIndex == 107]
    
        #Getting locations
        temp_df = temp_df[temp_df.location != 'Laser Enclosure']
        locations = temp_df.iloc[0]['location'].split(' ')
        temp_names = {}
        for i in range(len(locations)):
            temp_names[f'temperatureItem{i}'] = locations[i]
        temp_df.rename(columns=temp_names, inplace=True)
        temp_df.drop("LaserFan", axis=1, inplace=True)
        locations.remove('LaserFan')
    
        fig, (ax1, ax2) = plt.subplots(2,1,figsize=(15, 10))
        ret = ax1.plot(temp_df.index, temp_df[locations], label=locations)
        ax1.set_xlabel('Time (month/day - hour)')
        myFmt = mdates.DateFormatter('%m/%d - %H')
        #ax1.plot(df.index, dew_point, label='Dew Point')
        ax2.plot(hum_df.index, hum_df.relativeHumidityItem, label='Laser Rel. Humidity')
        ax2.set_ylabel('Relative Humidity')
        ax1.set_ylabel('Temperature (C)')
        ax1.legend(bbox_to_anchor=(1., 1.05))
        #ax1.set_ylim(5,20)
        plt.show()
    else:
        print(f'No Laser Temperature Data for {day_obs}')

In [ ]:
await get_laser_temp_plots(day_obs)

## Exposure Lists

### Electrometer Exposures

In [ ]:
day_df

### Calibration Exposures (LSSTCam)

In [ ]:
# This is a list of the test cases we regularly use for calibration products
test_cases = [90, 360, 556, 479, 563, 399, 566, 339]
test_cases = ['BLOCK-T'+str(test) for test in test_cases]

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all'], instrument='LSSTCam')

exposures = butler.query_dimension_records(
    "exposure", 
    where=f"instrument='LSSTCam' and exposure.day_obs={day_obs}"
)

# Build the table in one line
exposure_table = Table([exp.toDict() for exp in exposures])

# Filter by science_program
filtered = exposure_table[np.isin(exposure_table['science_program'], test_cases)]

result = filtered[
    'day_obs','group','physical_filter','obs_id',
    'exposure_time','dark_time','observation_reason',
    'science_program','seq_num','can_see_sky'
]

In [ ]:
result